In [2]:
#hide
#default_exp snip
#default_cls_lvl 3
from nbdev.showdoc import show_doc

In [4]:
#export
from nbdev.imports import *

# Snippets Manager

> How to manage snippets (json version)

In [5]:
show_doc(Config, title_level=3)

<h3 id="Config" class="doc_header"><code>class</code> <code>Config</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/foundation.py#L250" class="source_link" style="float:right">[source]</a></h3>

> <code>Config</code>(**`cfg_path`**, **`cfg_name`**, **`create`**=*`None`*)

Reading and writing `ConfigParser` ini files

In [1]:
# exporti
from glob import glob
import os
import time
import datetime
import pandas as pd
from io import StringIO
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display

def _getmtime_ms(path):
    mtime = os.path.getmtime(path)
    stamp = datetime.datetime.fromtimestamp(mtime, tz=datetime.timezone.utc)
    return int(round(stamp.timestamp()))


def _snippet_parser(path):
    sys.stdout.write(f"updating snippets from {path}\n")
    sys.stdout.flush()
    notebook = json.load(open(path))

    for cell in notebook["cells"]:
        if not cell["source"]:
            continue
        head = cell["source"][0]
        if head.startswith("# @"):
            desc, *tags = tuple(head.lstrip("# @").split("#"))
            sys.stdout.write(f"snippets: {desc}\n")
            sys.stdout.flush()
            tags = ";".join(tags) if tags else ""
            source = "".join(cell["source"][1:])
            url = ""
            yield (
                dict(
                    tags=tags,
                    desc=desc,
                    source=source,
                    url=url,
                )
            )

def _get_file_list(targets, exclude=["__pycache__", ".ipynb_checkpoints"]):
    """ "
    This function recursively prints all contents of a pathlib.Path object
    """
    import configparser

    nb_list = [(None, None)]
    for target in targets:
        target = Path(target)
        cfg1, cfg2 = target / "ghconfig", target / ".git/config"
        if cfg1.is_file():  # has ghconfig
            cfg = yaml.load(open(str(cfg1)), Loader=yaml.BaseLoader)
            root = cfg["repo"].rstrip(".git") + "/" + target.name
        elif cfg2.is_file():
            config = configparser.ConfigParser()
            config.read(str(cfg2))
            root = config['remote "origin"']["url"].rstrip(".git") + "/" + target.name
        else:
            root = None
        for e in exclude:
            if target.name.find(e) != -1:
                return list(map(list, zip(*nb_list)))
        for file in target.iterdir():
            if file.is_dir():
                if "." in file.name:
                    continue
                nb_list += _get_file_list([file.absolute().as_posix()], exclude=exclude)
            elif file.name.endswith(".ipynb"):
                nb_list.append((file.absolute().as_posix(), root))
    # remove invalid lines
    nb_list = list(filter(lambda x: x[1], nb_list))
    return list(map(list, zip(*nb_list)))

def _get_snippets_file_list(dirs):
    include = [d for d in dirs if not d.startswith("!")]
    exclude = [d[1:] for d in dirs if d.startswith("!")]
    nb_list, links = _get_file_list(include, exclude)
    mtime = list(map(os.path.getmtime, nb_list))
    return pd.DataFrame(zip(nb_list, mtime, links), columns=["path", "mtime", "url"])


def _disp(snippets, short=False):
    display(
        HTML(
            """
    <style>
    {pygments_css}
    </style>
    """.format(
                pygments_css=HtmlFormatter().get_style_defs(".highlight")
            )
        )
    )

    for _, row in snippets.iterrows():
        if short:
            content = row["desc"]
        else:
            content = row["desc"] + "\n" + row["source"][:80]
        display(
            HTML(data=highlight(f"({_})" + content, PythonLexer(), HtmlFormatter()))
        )

def _get_change_file(last, this):
    if not os.path.isfile(last):
        os.system(f'touch {last}')
    nb_list = glob('*.ipynb')
    mtime = list(map(_getmtime_ms, nb_list))
    df = (pd.DataFrame(zip(nb_list, mtime), columns=["path", "mtime"])
          .sort_values(by='path')
          .to_csv(this, sep='|', index=False, header=False))
#     print(f'diff -y --suppress-common-lines {last} {this}')
    diff_ret = os.popen(f'diff -y --suppress-common-lines {last} {this}').read()
    for line in diff_ret.splitlines():
        file = line.split('|')[0].split()[-1]
        yield file
    os.rename(this, last)

In [4]:
# exporti
@magics_class
class SnippetsMagics(Magics):
    """snip magic for snippets management.
    Provides the %snip magic.
    """
    def __init__(self, shell):
        super(SnippetsMagics, self).__init__(shell=shell)
        try:
            self.df = pd.read_json('snip.json')
        except ValueError:
            self.df = pd.DataFrame()
        
    @line_magic
    def sn(self, parameter_s="", last="last.txt", this='this.txt'):
        """snippets management

        Usage:
        * ``%snip``          - total numbers of all snippets and notebooks
        * ``%snip bar``      - search in the db
        * ``%snip -r``       - reset indexing
        * ``%snip -u``       - update db
        * ``%snip <key >-s``       - short output

        New search directories can be inserted into config.yaml['search_dirs']
        """
        opts, argsl = self.parse_options(parameter_s, "rust", mode="string")
        args = argsl.split()
        if "u" in opts: # update
            if "r" in opts:
                os.remove(last)
                os.remove('snip.json')
                self.df = pd.DataFrame()
            nestedList = map(_snippet_parser, list(_get_change_file(last, this)))
            ret_df = pd.DataFrame([item for sublist in nestedList for item in sublist])
            if self.df.empty:
                self.df = ret_df
            else:
#                 self.df = self.df.merge(ret_df, on='desc', how='right')
                self.df = pd.concat([self.df, ret_df], axis=0, join='outer')
            self.df.to_json('snip.json', orient='records')
        elif not args: # info
            try:
                n_ipynb = os.popen(f'wc -l {last}').read().split()[0]
            except:
                n_ipynb = 0
            print(
                "{} snippets, {} notebooks in SnippetDB".format(
                    self.df.shape[0], n_ipynb
                )
            )
        elif args[-1].isdigit():  # load snippets
            pos, q = int(args[-1]), " ".join(args[:-1])
            cell = (
                self.df.query(f"desc.str.contains('{q}')", engine="python")
                .pipe(lambda _df: _df.iloc[pos]["source"]))
            self.shell.set_next_input(cell, replace=False)
        else:
            q = " ".join(args)
            (
                self.df.query(f"desc.str.contains('{q}')", engine="python")
                .reset_index()
                .pipe(_disp, short="s" in opts)
            )

def load_ipython_extension(ipython):
    ipython.register_magics(SnippetsMagics)

In [ ]:
def load_ipython_extension(ipython):
    ipython.register_magics(SnippetsMagics)

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
!wc -l last.snapshot.txt

       4 last.snapshot.txt


In [53]:
'       4 last.snapshot.txt'.split()

['4', 'last.snapshot.txt']

In [72]:
%reload_ext sniptool

In [73]:
%sn -u -r

updating snippets from 00_core.ipynb
snippets: check package version 
snippets: test

updating snippets from 01_viz.ipynb
snippets: seaborn set1  
snippets: dataset alta 
updating snippets from 02_snip.ipynb
updating snippets from index.ipynb


In [75]:
%sn

4 snippets, 4 notebooks in SnippetDB


In [10]:
!nbdev_build_lib --fname 01_viz.ipynb
!nbdev_build_lib --fname 02_snip.ipynb

Converted 01_viz.ipynb.
Converted 02_snip.ipynb.


In [ ]:
!nbdev_update_lib 